In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
df = pd.read_csv('labels.csv')
df.head()

,filename,xmin,xmax,ymin,ymax
0,./images\N1.xml,1093,1396,645,727
1,./images\N100.xml,134,301,312,350
2,./images\N101.xml,31,139,128,161
3,./images\N102.xml,164,316,216,243
4,./images\N103.xml,813,1067,665,724


In [4]:
import xml.etree.ElementTree as xet

In [5]:
filename = df['filename'][0]
filename

'./images\\N1.xml'

In [6]:
def getfilename(filename):
	filename_image = xet.parse(filename).getroot().find('filename').text
	filepath_image = os.path.join('./images',filename_image)
	return filepath_image

In [19]:
images_path = list(df['filename'].apply(getfilename))
images_path

['./images\\N1.jpeg',
 './images\\N100.jpeg',
 './images\\N101.jpeg',
 './images\\N102.jpeg',
 './images\\N103.jpeg',
 './images\\N104.jpeg',
 './images\\N105.jpeg',
 './images\\N106.jpeg',
 './images\\N107.jpeg',
 './images\\N108.jpeg',
 './images\\N109.jpeg',
 './images\\N11.jpeg',
 './images\\N110.jpeg',
 './images\\N111.jpeg',
 './images\\N112.jpeg',
 './images\\N113.jpeg',
 './images\\N114.jpeg',
 './images\\N116.jpeg',
 './images\\N117.jpeg',
 './images\\N118.jpeg',
 './images\\N119.jpeg',
 './images\\N12.jpeg',
 './images\\N120.jpeg',
 './images\\N121.jpeg',
 './images\\N122.jpeg',
 './images\\N123.jpeg',
 './images\\N124.jpeg',
 './images\\N126.jpeg',
 './images\\N127.jpeg',
 './images\\N128.jpeg',
 './images\\N129.jpeg',
 './images\\N130.jpeg',
 './images\\N131.jpeg',
 './images\\N132.jpeg',
 './images\\N133.jpeg',
 './images\\N134.jpeg',
 './images\\N135.jpeg',
 './images\\N136.jpeg',
 './images\\N137.jpeg',
 './images\\N138.jpeg',
 './images\\N139.jpeg',
 './images\\N14.jpeg

# to verify image 'n' output are in sync

In [8]:
file_path = images_path[0]
file_path

'./images\\N1.jpeg'

In [9]:
img = cv2.imread(file_path)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [10]:
cv2.rectangle(img,(1094,645),(1396,727),(0,255,0),3)
cv2.namedWindow('example',cv2.WINDOW_NORMAL)
cv2.imshow('example',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# data processing

In [16]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [17]:
labels = df.iloc[ : ,1:].values

In [31]:
data = []
output = []
for ind in range (len(images_path)):
    image = images_path[ind]
    img_arr = cv2.imread(image)
    h,w,d=img_arr.shape
    #preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # normalization
    #normalisation to labels
    xmin , xmax , ymin , ymax = labels[ind]
    nxmin, nxmax = xmin/w, xmax/w
    nymin, nymax = ymin/h, ymax/h
    labels_norm = (nxmin, nxmax, nymin, nymax)
    #------------------append
    data.append(norm_load_image_arr)
    output.append(labels_norm)

In [25]:
labels_norm

(0.5692708333333333,
 0.7270833333333333,
 0.5972222222222222,
 0.6731481481481482)

In [32]:
load_image_arr

array([[[45., 45., 43.],
        [43., 43., 41.],
        [57., 57., 55.],
        ...,
        [52., 53., 48.],
        [55., 56., 51.],
        [59., 60., 55.]],

       [[44., 44., 42.],
        [50., 50., 48.],
        [56., 56., 54.],
        ...,
        [51., 52., 47.],
        [53., 54., 49.],
        [55., 56., 51.]],

       [[49., 49., 47.],
        [49., 49., 47.],
        [50., 50., 48.],
        ...,
        [53., 54., 49.],
        [54., 55., 50.],
        [53., 54., 49.]],

       ...,

       [[14., 25., 55.],
        [16., 27., 57.],
        [17., 28., 58.],
        ...,
        [67., 62., 56.],
        [64., 59., 53.],
        [65., 60., 54.]],

       [[13., 26., 58.],
        [18., 31., 63.],
        [19., 32., 64.],
        ...,
        [56., 51., 45.],
        [55., 50., 44.],
        [55., 50., 44.]],

       [[15., 33., 71.],
        [16., 35., 75.],
        [17., 36., 76.],
        ...,
        [49., 46., 41.],
        [45., 42., 37.],
        [50., 47., 42.]]

In [38]:
X = np.array(data, dtype=np.float32)
y = np.array(output, dtype=np.float32)

In [39]:
X.shape,y.shape

((225, 224, 224, 3), (225, 4))

In [41]:
xtrain,xtest,ytrain,ytest = train_test_split(X,y,train_size=0.8,random_state=0)
xtrain.shape,xtest.shape,ytrain.shape,ytest.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

# DEEP LEARNING

In [55]:
from tensorflow.keras.applications import MobileNetV2, InceptionV3, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [58]:
inception_res_net = InceptionResNetV2(weights="imagenet" ,include_top=False,
                                        input_tensor=Input(shape = (224,224,3)))
inception_res_net.trainable=False
#------------------------
headmodel = inception_res_net.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500, activation= "relu") (headmodel)
headmodel = Dense(250, activation= "relu") (headmodel)
headmodel = Dense(4, activation= 'sigmoid') (headmodel)
#------------------------
model = Model(inputs=inception_res_net.input, outputs=headmodel)


In [60]:
# complie model
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_406 (Conv2D)            (None, 111, 111, 32  864         ['input_10[0][0]']               
                                )                                                                 
                                                                                                  
 batch_normalization_406 (Batch  (None, 111, 111, 32  96         ['conv2d_406[0][0]']             
 Normalization)                 )                                                             

 batch_normalization_411 (Batch  (None, 25, 25, 96)  288         ['conv2d_411[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_413 (Batch  (None, 25, 25, 64)  192         ['conv2d_413[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_416 (Batch  (None, 25, 25, 96)  288         ['conv2d_416[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_417 (Batch  (None, 25, 25, 64)  192         ['conv2d_417[0][0]']             
 Normaliza

 activation_427 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_427[0][0]']
                                                                                                  
 conv2d_425 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_428 (Conv2D)            (None, 25, 25, 48)   13824       ['activation_427[0][0]']         
                                                                                                  
 batch_normalization_425 (Batch  (None, 25, 25, 32)  96          ['conv2d_425[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_428 (Batch  (None, 25, 25, 48)  144         ['conv2d_428[0][0]']             
 Normaliza

                                                                                                  
 activation_430 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_430[0][0]']
                                                                                                  
 activation_432 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_432[0][0]']
                                                                                                  
 activation_435 (Activation)    (None, 25, 25, 64)   0           ['batch_normalization_435[0][0]']
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 25, 25, 128)  0           ['activation_430[0][0]',         
                                                                  'activation_432[0][0]',         
                                                                  'activation_435[0][0]']         
          

 Normalization)                                                                                   
                                                                                                  
 batch_normalization_446 (Batch  (None, 25, 25, 48)  144         ['conv2d_446[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_443 (Activation)    (None, 25, 25, 32)   0           ['batch_normalization_443[0][0]']
                                                                                                  
 activation_446 (Activation)    (None, 25, 25, 48)   0           ['batch_normalization_446[0][0]']
                                                                                                  
 conv2d_442 (Conv2D)            (None, 25, 25, 32)   10240       ['block35_4_ac[0][0]']           
          

...

 batch_normalization_575 (Batch  (None, 5, 5, 224)   672         ['conv2d_575[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_575 (Activation)    (None, 5, 5, 224)    0           ['batch_normalization_575[0][0]']
                                                                                                  
 conv2d_573 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_576 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_575[0][0]']         
                                                                                                  
 batch_normalization_573 (Batch  (None, 5, 5, 192)   576         ['conv2d_573[0][0]']             
 Normaliza

 conv2d_584 (Conv2D)            (None, 5, 5, 256)    172032      ['activation_583[0][0]']         
                                                                                                  
 batch_normalization_581 (Batch  (None, 5, 5, 192)   576         ['conv2d_581[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_584 (Batch  (None, 5, 5, 256)   768         ['conv2d_584[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_581 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_581[0][0]']
                                                                                                  
 activatio

                                                                                                  
 activation_589 (Activation)    (None, 5, 5, 192)    0           ['batch_normalization_589[0][0]']
                                                                                                  
 activation_592 (Activation)    (None, 5, 5, 256)    0           ['batch_normalization_592[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 5, 5, 448)    0           ['activation_589[0][0]',         
                                                                  'activation_592[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 

                                                                                                  
 block8_8_conv (Conv2D)         (None, 5, 5, 2080)   933920      ['block8_8_mixed[0][0]']         
                                                                                                  
 block8_8 (Lambda)              (None, 5, 5, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 5, 5, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_602 (Conv2D)            (None, 5, 5, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_nor

                                                                                                  
 conv_7b_bn (BatchNormalization  (None, 5, 5, 1536)  4608        ['conv_7b[0][0]']                
 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 5, 5, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 flatten_1 (Flatten)            (None, 38400)        0           ['conv_7b_ac[0][0]']             
                                                                                                  
 dense_3 (Dense)                (None, 500)          19200500    ['flatten_1[0][0]']              
                                                                                                  
 dense_4 (

# MODEL TRAINING

In [61]:
from tensorflow.keras.callbacks import TensorBoard

In [62]:
tfb = TensorBoard('object_detection')

In [66]:
history = model.fit(x=xtrain,y=ytrain,batch_size=17,epochs=100,validation_data=(xtest,ytest),callbacks=[tfb])

Epoch 1/100
11/11 [==============================] - 73s 5s/step - loss: 0.1376 - val_loss: 0.1314
Epoch 2/100
11/11 [==============================] - 53s 5s/step - loss: 0.0907 - val_loss: 0.0820
Epoch 3/100
11/11 [==============================] - 51s 5s/step - loss: 0.0673 - val_loss: 0.0744
Epoch 4/100
11/11 [==============================] - 51s 5s/step - loss: 0.0515 - val_loss: 0.0621
Epoch 5/100
11/11 [==============================] - 53s 5s/step - loss: 0.0462 - val_loss: 0.0601
Epoch 6/100
11/11 [==============================] - 56s 5s/step - loss: 0.0442 - val_loss: 0.0602
Epoch 7/100
11/11 [==============================] - 53s 5s/step - loss: 0.0417 - val_loss: 0.0598
Epoch 8/100
11/11 [==============================] - 53s 5s/step - loss: 0.0426 - val_loss: 0.0621
Epoch 9/100
11/11 [==============================] - 53s 5s/step - loss: 0.0455 - val_loss: 0.0641
Epoch 10/100
11/11 [==============================] - 55s 5s/step - loss: 0.0438 - val_loss: 0.0581
Epoch 11/

11/11 [==============================] - 66s 6s/step - loss: 0.0370 - val_loss: 0.0548
Epoch 84/100
11/11 [==============================] - 68s 6s/step - loss: 0.0370 - val_loss: 0.0551
Epoch 85/100
11/11 [==============================] - 64s 6s/step - loss: 0.0374 - val_loss: 0.0556
Epoch 86/100
11/11 [==============================] - 65s 6s/step - loss: 0.0385 - val_loss: 0.0553
Epoch 87/100
11/11 [==============================] - 68s 6s/step - loss: 0.0380 - val_loss: 0.0549
Epoch 88/100
11/11 [==============================] - 66s 6s/step - loss: 0.0375 - val_loss: 0.0547
Epoch 89/100
11/11 [==============================] - 63s 6s/step - loss: 0.0373 - val_loss: 0.0549
Epoch 90/100
11/11 [==============================] - 58s 5s/step - loss: 0.0372 - val_loss: 0.0550
Epoch 91/100
11/11 [==============================] - 58s 5s/step - loss: 0.0371 - val_loss: 0.0550
Epoch 92/100
11/11 [==============================] - 59s 5s/step - loss: 0.0371 - val_loss: 0.0546
Epoch 93/100


In [67]:
history = model.fit(x=xtrain,y=ytrain,batch_size=17,epochs=150,validation_data=(xtest,ytest),callbacks=[tfb],initial_epoch=100)

Epoch 101/150
11/11 [==============================] - 50s 5s/step - loss: 0.0369 - val_loss: 0.0544
Epoch 102/150
11/11 [==============================] - 48s 4s/step - loss: 0.0339 - val_loss: 0.0566
Epoch 103/150
11/11 [==============================] - 49s 5s/step - loss: 0.0262 - val_loss: 0.0122
Epoch 104/150
11/11 [==============================] - 50s 5s/step - loss: 0.0142 - val_loss: 0.0125
Epoch 105/150
11/11 [==============================] - 51s 5s/step - loss: 0.0100 - val_loss: 0.0089
Epoch 106/150
11/11 [==============================] - 53s 5s/step - loss: 0.0078 - val_loss: 0.0103
Epoch 107/150
11/11 [==============================] - 54s 5s/step - loss: 0.0062 - val_loss: 0.0086
Epoch 108/150
11/11 [==============================] - 56s 5s/step - loss: 0.0051 - val_loss: 0.0068
Epoch 109/150
11/11 [==============================] - 56s 5s/step - loss: 0.0034 - val_loss: 0.0080
Epoch 110/150
11/11 [==============================] - 53s 5s/step - loss: 0.0028 - val_los

In [68]:
model.save('./models/object_detection.h5')